In [52]:
import torch
from torch import nn, optim
import sys
sys.path.append('../')
#from tta_agg_models import TTARegression, TTAPartialRegression, GPS
from utils.gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('0')
class TTAPartialRegression(nn.Module):
    def __init__(self, n_augs, n_classes, temp_scale=1, initialization='even',coeffs=[]):
        super().__init__()
        # To make "a" and "b" real parameters of the model, we need to wrap them with nn.Parameter
        self.coeffs = nn.Parameter(torch.randn((n_augs,1 ), requires_grad=True, dtype=torch.float))
        self.temperature = temp_scale
        if len(coeffs):
            self.coeffs = nn.Parameter(torch.Tensor(coeffs), requires_grad=True)
        else:
            if initialization == 'even':
                self.coeffs.data.fill_(1.0/n_augs)
            elif initialization== 'original':
                self.coeffs.data[0,:].fill_(1)
                self.coeffs.data[1,:].fill_(0)

    def forward(self, x):
        # Computes the outputs / predictions
        x = x/self.temperature
        mult = torch.matmul(x.transpose(1, 2), self.coeffs / torch.sum(self.coeffs, axis=0))
        return mult.squeeze()

    
dataset = 'birds200'
n_augs = 200
n_classes = 200    
agg_models_dir = '../' + dataset + '/pil/agg_models'
model_name = 'resnet50'
aug_name = 'combo'
model_path = agg_models_dir + '/'+model_name+'/'+aug_name + '/partial_lr.pth'

model = TTAPartialRegression(n_augs,n_classes,1,'even')
model.load_state_dict(torch.load(model_path))
model.eval()

Using GPU:0


TTAPartialRegression()

In [53]:
coeffs = model.coeffs.detach().cpu().numpy()

In [54]:
import numpy as np
from utils.aug_utils import invert_aug_list
aug_list = np.load('../' + 'flowers102' + '/' + 'five_crop_hflip_scale' + '/aug_list.npy')
aug_order = np.load('../' +'flowers102' + '/' + 'five_crop_hflip_scale' + '/aug_order.npy')
aug_names = invert_aug_list(aug_list, aug_order)

In [55]:
[aug_names[i] for i in np.where(coeffs == 0)[0]]

IndexError: list index out of range

In [56]:
[aug_names[i] for i in np.where(coeffs != 0)[0]]

IndexError: list index out of range

In [57]:
coeffs

array([[0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.04704392],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.02798502],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.00645093],
       [0.00645093],
       [0.00645093],
       [0.00645093],
       [0.00645093],
       [0.00645093],
       [0.00645093],
       [0.006